In [ ]:

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from networkx import shortest_path_length
import json
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

class ProductRecommendationSystem:
    def __init__(self):
        self.graph = nx.DiGraph()
        self.description_model = SentenceTransformer("dragonkue/bge-m3-ko")  # 모델 로드


    def compute_description_similarity(self, target_embedding, candidate_embedding):
        """사전 계산된 임베딩을 활용한 유사도 계산"""
        return cosine_similarity([target_embedding], [candidate_embedding])[0, 0]
        
    def create_category_graph(self, data_path):
        """카테고리 계층 구조 그래프를 생성합니다."""
        data = pd.read_csv(data_path)
        categories = data[['카테고리1', '카테고리2', '카테고리3', '카테고리4']]
        categories = categories.replace('None', pd.NA)
        
        # 카테고리 경로 생성 및 그래프에 추가
        category_paths = categories.apply(lambda x: '/'.join(x.dropna().astype(str)), axis=1)
        category_paths = category_paths.apply(lambda x: x.split('/'))
        
        for path in category_paths:
            for i in range(len(path) - 1):
                self.graph.add_edge(path[i], path[i + 1])
        
        # Root 노드 추가
        if 'Root' not in self.graph:
            self.graph.add_node('Root')
            for node in self.graph.nodes():
                if node != 'Root' and not list(self.graph.predecessors(node)):
                    self.graph.add_edge('Root', node)
                    
    def visualize_category_hierarchy(self, output_path="category_hierarchy.png"):
        """카테고리 계층 구조를 시각화합니다."""
        plt.figure(figsize=(15, 10))
        pos = nx.spring_layout(self.graph)
        nx.draw(self.graph, pos, with_labels=True, node_size=1500, 
                node_color="skyblue", font_size=10, font_weight="bold")
        plt.savefig(output_path)
        plt.close()
        


    def wu_palmer_similarity(self, category_path1, category_path2):
        """카테고리 계층 구조에서 Wu-Palmer 유사도를 계산합니다."""
        try:
            # 두 경로가 동일하면 유사도 1 반환
            if category_path1 == category_path2:
                return 1.0

            # 공통 조상의 깊이를 찾기
            common_depth = 0
            for i in range(min(len(category_path1), len(category_path2))):
                if category_path1[i] == category_path2[i]:
                    common_depth = i + 1
                else:
                    break

            # 공통 조상이 "음식"같이 최상위일 경우 유사도를 낮게 계산하도록 유도
            if common_depth == 0:
                return 0  # 공통 조상이 없는 경우

            # 각 경로의 깊이를 구합니다
            depth1 = len(category_path1)
            depth2 = len(category_path2)

            # Wu-Palmer 유사도 공식 적용
            similarity = (2 * common_depth) / (depth1 + depth2)
            return similarity

        except nx.NodeNotFound:
            return 0
    
    def create_user_purchase_history(self, orders_paths, item_profiles):
        """사용자별 구매 이력을 생성합니다."""
        user_purchase_history = defaultdict(list)
        user_item_history = defaultdict(list)
        
        for orders_path in orders_paths:
            df = pd.read_csv(orders_path)
            
            for _, row in df.iterrows():
                customer_id = row['고객식별ID']
                item_seq = str(row['상품ID'])
                
                if item_seq in item_profiles:
                    categories = item_profiles[item_seq]['categories']
                    description = item_profiles[item_seq].get('description', '')  # 상품 설명 가져오기
                    
                    if categories and categories[0]:
                        # 구매 이력에 카테고리와 설명 추가
                        user_purchase_history[customer_id].append({
                            'categories': categories,
                            'description': description
                        })
                        # 구매한 상품 ID 기록
                        user_item_history[customer_id].append(item_seq)
        
        return dict(user_purchase_history), dict(user_item_history)
    
    

    def recommend_products(self, user_purchase_history, user_item_history, item_profiles, target_user, top_n=10, number_filter=None):
        """상품을 추천합니다."""
        recommendations = []

        # target_user의 구매 이력 가져오기
        user_history = user_item_history.get(target_user, [])

        candidate_items = set()  # 후보군 상품 ID를 저장

        # Step 1: 후보군을 필터링하고 후보군 상품 ID 수집
        for item_id, item_data in item_profiles.items():
            item_name = item_data.get('name')
            item_categories = item_data['categories']
            item_description = item_data.get('description')

            # 상품 이름이 해당 숫자로 끝나는지 확인
            if number_filter and not item_name.endswith(number_filter):
                continue 

            if not item_name or not item_categories or not item_description:
                continue

            # 후보군 상품 ID 추가
            candidate_items.add(item_id)

        # Step 2: 후보군에서 유사도 계산
        for item_id in candidate_items:
            item_data = item_profiles[item_id]
            item_name = item_data.get('name')
            item_categories = item_data['categories']
            item_description = item_data.get('description')

            max_category_similarity = 0
            max_description_similarity = 0

            # target_user가 구매한 아이템들과 비교
            for purchased_item_id in user_history:
                if purchased_item_id not in item_profiles:
                    continue  # item_profiles에 없는 경우 건너뜁니다.

                purchased_item_data = item_profiles[purchased_item_id]
                purchased_categories = purchased_item_data['categories']
                purchased_description_embedding = purchased_item_data.get('embedding')

                # 카테고리 유사도 계산
                if all(node in self.graph for node in item_categories) and \
                all(node in self.graph for node in purchased_categories):
                    category_similarity = self.wu_palmer_similarity(item_categories, purchased_categories)
                    max_category_similarity = max(max_category_similarity, category_similarity)

                # 상품 설명 유사도 계산
                if purchased_description_embedding:
                    description_similarity = self.compute_description_similarity(
                        item_data.get('embedding'), purchased_description_embedding
                    )
                    max_description_similarity = max(max_description_similarity, description_similarity)

            # 추천 리스트에 추가
            recommendations.append({
                'item_id': item_id,
                'name': item_name,
                'category_similarity': max_category_similarity,
                'description_similarity': max_description_similarity
            })

        # Step 3: 정렬 후 반환
        recommendations.sort(key=lambda x: (x['category_similarity'], x['description_similarity']), reverse=True)
        return recommendations[:top_n]


    


def get_category_path_string(categories):
    """카테고리 리스트를 문자열로 변환합니다."""
    return ' > '.join(categories) if categories else ''

def load_item_profiles(json_path):
    """JSON 파일에서 item_profiles를 불러옵니다."""
    with open(json_path, 'r', encoding='utf-8') as f:
        item_profiles = json.load(f)
    return item_profiles

def main():
    json_path = 'item_profiles_with_embeddings.json'
    item_profiles = load_item_profiles(json_path)
    
    recommender = ProductRecommendationSystem()
    recommender.create_category_graph("left_joined_file.csv")
    
    orders_paths = ['order1_df.csv', 'order2_df.csv', 'order3_df.csv', 'order4_df.csv', 'order5_df.csv']
    purchase_history, item_history = recommender.create_user_purchase_history(orders_paths, item_profiles)
    
    # 사용자 입력 추가
    user_id = input("추천을 받을 사용자 ID를 입력하세요: ")
    number_filter = input("필터링할 숫자를 입력하세요 (1-5 중 하나): ")


    print(f"사용자 {user_id}의 구매 이력:")
    print('-' * 40)

    for item_id in item_history[user_id]:
        category_path = item_profiles[item_id]['categories']
        item_name = item_profiles[item_id]['name']
        print(f"상품 이름: {item_name}")
        print(f"카테고리: {get_category_path_string(category_path)}")

       

    if user_id in purchase_history:
        print(f"\n{'='*80}")
        print(f"사용자 {user_id}의 추천 상품 (필터링 기준: 이름이 '{number_filter}'로 끝나는 상품):")
        recommendations = recommender.recommend_products(
            purchase_history, item_history, item_profiles, user_id, number_filter=number_filter
        )
        
        for rec in recommendations:
            print(f"상품 이름: {rec['name']}")
            print(f"카테고리: {get_category_path_string(item_profiles[rec['item_id']]['categories'])}")
            print(f"카테고리 유사도: {rec['category_similarity']:.4f}")
            print(f"설명 유사도: {rec['description_similarity']:.4f}")
            print('-' * 40)
    else:
        print("입력한 사용자 ID에 대한 구매 이력이 없습니다.")



        

if __name__ == "__main__":
    main()